In [5]:
"""
Starter code for the problem "Hamilton-Jacobi reachability".

Autonomous Systems Lab (ASL), Stanford University
"""

import os

import jax
import jax.numpy as jnp
import numpy as np

import hj_reachability as hj

import matplotlib.pyplot as plt
from animations import animate_planar_quad


In [10]:
9.807 * 2.5 * 0.75

18.388125000000002

In [102]:
# Define problem ingredients (exercise parts (a), (b), (c)).


class PlanarQuadrotor:

    def __init__(self):
        # Dynamics constants
        # yapf: disable
        self.g = 9.807         # gravity (m / s**2)
        self.m = 2.5           # mass (kg)
        self.l = 1.0           # half-length (m)
        self.Iyy = 1.0         # moment of inertia about the out-of-plane axis (kg * m**2)
        self.Cd_v = 0.25       # translational drag coefficient
        self.Cd_phi = 0.02255  # rotational drag coefficient
        # yapf: enable

        # Control constraints
        self.max_thrust_per_prop = (
            0.75 * self.m * self.g
        )  # total thrust-to-weight ratio = 1.5
        self.min_thrust_per_prop = (
            0  # at least until variable-pitch quadrotors become mainstream :D
        )

    def full_dynamics(self, full_state, control):
        """Continuous-time dynamics of a planar quadrotor expressed as an ODE."""
        x, v_x, y, v_y, phi, omega = full_state
        T_1, T_2 = control
        return jnp.array(
            [
                v_x,
                (-(T_1 + T_2) * jnp.sin(phi) - self.Cd_v * v_x) / self.m,
                v_y,
                ((T_1 + T_2) * jnp.cos(phi) - self.Cd_v * v_y) / self.m - self.g,
                omega,
                ((T_2 - T_1) * self.l - self.Cd_phi * omega) / self.Iyy,
            ]
        )

    def dynamics(self, state, control):
        """Reduced (for the purpose of reachable set computation) continuous-time dynamics of a planar quadrotor."""
        y, v_y, phi, omega = state
        T_1, T_2 = control
        return jnp.array(
            [
                v_y,
                ((T_1 + T_2) * jnp.cos(phi) - self.Cd_v * v_y) / self.m - self.g,
                omega,
                ((T_2 - T_1) * self.l - self.Cd_phi * omega) / self.Iyy,
            ]
        )

    def optimal_control(self, state, grad_value):
        """Computes the optimal control realized by the HJ PDE Hamiltonian.

        Args:
            state: An unbatched (!) state vector, an array of shape `(4,)` containing `[y, v_y, phi, omega]`.
            grad_value: An array of shape `(4,)` containing the gradient of the value function at `state`.

        Returns:
            A vector of optimal controls, an array of shape `(2,)` containing `[T_1, T_2]`, that minimizes
            `grad_value @ self.dynamics(state, control)`.
        """
        # PART (a): WRITE YOUR CODE BELOW ###############################################
        # You may find `jnp.where` to be useful; see corresponding numpy docstring:
        # https://numpy.org/doc/stable/reference/generated/numpy.where.html
        n = 100

        # T1_grid = np.linspace(self.min_thrust_per_prop,self.max_thrust_per_prop, n)
        # T2_grid = np.linspace(self.min_thrust_per_prop,self.max_thrust_per_prop, n)
        # H_grid = np.zeros((n, n))

        
        # T1_grid = [self.min_thrust_per_prop,self.max_thrust_per_prop]
        # T2_grid = [self.min_thrust_per_prop,self.max_thrust_per_prop]
        # H_grid = np.zeros((2, 2))
        # H_grid = [] #np.zeros((2, 2))
        
        # # state = np.zeros((4, 1))
        # # grad_value = np.zeros((4, 1))
        # control = jnp.array([self.min_thrust_per_prop, self.min_thrust_per_prop]).astype(float)
        # H_min = jnp.inf
        # for i, T1 in enumerate(T1_grid):
        #     for j, T2 in enumerate(T2_grid):
        #         # print(grad_value.shape, self.dynamics(state, [T1, T2]).shape)
        #         f = self.dynamics(state, [T1, T2])
        #         # print(f, grad_value, jnp.dot(f, grad_value))
        #         # break
        #         # H_grid[i,j] = jnp.dot(grad_value,  self.dynamics(state, [T1, T2]))
        #         H = jnp.dot(grad_value,  self.dynamics(state, [T1, T2]))
                
        #         H_grid.append(jnp.dot(grad_value,  self.dynamics(state, [T1, T2])))
        control = jnp.where(
            jnp.array(
                [
                    jnp.dot(grad_value,  self.dynamics(state, [self.min_thrust_per_prop, self.min_thrust_per_prop])) < \
                    jnp.dot(grad_value,  self.dynamics(state, [self.max_thrust_per_prop, self.min_thrust_per_prop])), 
                    jnp.dot(grad_value,  self.dynamics(state, [self.min_thrust_per_prop, self.min_thrust_per_prop])) < \
                    jnp.dot(grad_value,  self.dynamics(state, [self.min_thrust_per_prop, self.max_thrust_per_prop])), 
                ]
            ),
            jnp.array([self.min_thrust_per_prop,self.min_thrust_per_prop]),
            jnp.array([self.max_thrust_per_prop,self.max_thrust_per_prop]),
        )
        return control
            # break

        # H_grid = jnp.array(H_grid)
        
        # i_min, j_min = jnp.unravel_index(H_grid.argmin(), (2,2))

        # # print(jnp.array([T1_grid[i_min], T2_grid[j_min]]))
                
        # return jnp.array([T1_grid[i_min], T2_grid[j_min]]).astype(float)


        #  [T1_grid[i_min], T2_grid[j_min]] = jnp.where(conditions, [min min], [max max], [max min], [max ])
        # raise NotImplementedError
        #################################################################################

    def hamiltonian(self, state, time, value, grad_value):
        """Evaluates the HJ PDE Hamiltonian."""
        del time, value  # unused
        control = self.optimal_control(state, grad_value)
        return grad_value @ self.dynamics(state, control)

    def partial_max_magnitudes(self, state, time, value, grad_value_box):
        """Computes the max magnitudes of the Hamiltonian partials over the `grad_value_box` in each dimension."""
        del time, value, grad_value_box  # unused
        y, v_y, phi, omega = state
        return jnp.array(
            [
                jnp.abs(v_y),
                (
                    2 * self.max_thrust_per_prop * jnp.abs(jnp.cos(phi))
                    + self.Cd_v * jnp.abs(v_y)
                )
                / self.m
                + self.g,
                jnp.abs(omega),
                (
                    (self.max_thrust_per_prop - self.min_thrust_per_prop) * self.l
                    + self.Cd_phi * jnp.abs(omega)
                )
                / self.Iyy,
            ]
        )

In [103]:
def test_optimal_control(n=10, seed=0):
    planar_quadrotor = PlanarQuadrotor()
    optimal_control = jax.jit(planar_quadrotor.optimal_control)# 
    np.random.seed(seed)
    states = 5 * np.random.normal(size=(n, 4))
    grad_values = np.random.normal(size=(n, 4))
    try:
        for state, grad_value in zip(states, grad_values):
            if not jnp.issubdtype(
                optimal_control(state, grad_value).dtype, jnp.floating
            ):
                raise ValueError(
                    "`PlanarQuadrotor.optimal_control` must return a `float` array (i.e., not `int`)."
                )
            opt_hamiltonian_value = grad_value @ planar_quadrotor.dynamics(
                state, optimal_control(state, grad_value)
            )
            for T_1 in (
                planar_quadrotor.min_thrust_per_prop,
                planar_quadrotor.max_thrust_per_prop,
            ):
                for T_2 in (
                    planar_quadrotor.min_thrust_per_prop,
                    planar_quadrotor.max_thrust_per_prop,
                ):
                    hamiltonian_value = grad_value @ planar_quadrotor.dynamics(
                        state, np.array([T_1, T_2])
                    )
                    if opt_hamiltonian_value > hamiltonian_value + 1e-4:
                        raise ValueError(
                            "Check your logic for `PlanarQuadrotor.optimal_control`; with "
                            f"`state` {state} and `grad_value` {grad_value}, got optimal control"
                            f"{optimal_control(state, grad_value)} with corresponding Hamiltonian value "
                            f"{opt_hamiltonian_value:7.4f} but {np.array([T_1, T_2])} has a lower corresponding "
                            f"value {hamiltonian_value:7.4f}."
                        )
    except (jax.errors.JAXTypeError, jax.errors.JAXIndexError, AssertionError) as e:
        print(
            "`PlanarQuadrotor.optimal_control` must be implemented using only `jnp` operations; "
            "`np` may only be used for constants, "
            "and `jnp.where` must be used instead of native python control flow (`if`/`else`)."
        )
        raise e
test_optimal_control()
# test_target_set()
# test_envelope_set()


In [ ]:
state = np.ar

In [49]:
max_thrust_per_prop = 18.388125000000002
min_thrust_per_prop = 0
n = 100
T1_grid = np.linspace(min_thrust_per_prop,max_thrust_per_prop, n)
T2_grid = np.linspace(min_thrust_per_prop,max_thrust_per_prop, n)
H_grid = np.zeros((n, n))
state = np.zeros((4, 1))
grad_value = np.zeros((4, 1))
for i, T1 in T1_grid:
    for j, T2 in T2_grid:
        H_grid[i,j] = grad_value.T @ dynamics(state, (T1, T2))

TypeError: cannot unpack non-iterable numpy.float64 object

In [30]:
A = np.random.rand(2, 2)
print(A)
np.unravel_index(A.argmin(), A.shape)
# np.where(np.min(mat)==mat, mat)

[[0.65210327 0.43141844]
 [0.8965466  0.36756187]]


(1, 1)

In [52]:
np.dot([1,1,1,1], [1,2,3,4])

10

In [7]:
def target_set(state):
    """A real-valued function such that the zero-sublevel set is the target set.

    Args:
        state: An unbatched (!) state vector, an array of shape `(4,)` containing `[y, v_y, phi, omega]`.

    Returns:
        A scalar, nonpositive iff the state is in the target set.
    """
    # PART (b): WRITE YOUR CODE BELOW ###############################################
    raise NotImplementedError
    #################################################################################

In [8]:
def envelope_set(state):
    """A real-valued function such that the zero-sublevel set is the operational envelope.

    Args:
        state: An unbatched (!) state vector, an array of shape `(4,)` containing `[y, v_y, phi, omega]`.

    Returns:
        A scalar, nonpositive iff the state is in the operational envelope.
    """
    # PART (c): WRITE YOUR CODE BELOW ###############################################
    raise NotImplementedError
    #################################################################################